In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import plotly.offline as offline

offline.init_notebook_mode(connected=True)

## Animations
Animations in Plotly involve defining a sequence of frames which will be animated on on the plot. When defined correctly, it can be made to look like a moving plot.

We will plot a line in this demo and animate it so that it appears as though the line is being drawn.

#### Define the starting point for the animation
Our animation will involve starting from a point (0,11)

In [2]:
x0 = [0]
y0 = [11]

#### Define the second frame
The second frame in our animation will involve a line being plotted from the first point to the second point. For that we need to include the point from the first frame and add the point from the second frame

In [3]:
x1 = [0, 1]
y1 = [11, 13]

#### The third frame
This will start from the 2nd frame and add a third point to which the line will extend in the animation

In [4]:
x2 = [0, 1, 2]
y2 = [11, 13, 12]

#### Define the layout
If you don't specify the range for the X and Y axes, then the plot will dynamically adjust the range of the axes on display according to the points which have been plotted until the current frame. This will cause a lot of dynamic resizing of the axes which makes the animation very distracting.

We specify the range for the X and Y axes to include all the points which will be plotted during the animation. 

Under the updatemenus field, we can define buttons which we would like. Here, we add a play button. Note that for the args, we pass in an empty list, which means that all frames will be animated.

In [5]:
layout = go.Layout(xaxis = dict(range = [0, 3]), 
                   yaxis = dict(range = [10, 15]),
                   
                   updatemenus = [dict(type = 'buttons',
                                       buttons = [dict(label = 'Play',
                                                       method= 'animate',
                                                       args= [None]
                                                       )
                                                  ]
                                       )
                                 ]
                  )

#### Define the frames included in our plot
We configure a set of frames which will be plotted in our animated figure

In [6]:
frames = go.Frames([dict(data = [dict(x = x0, y = y0)]), 
                    dict(data = [dict(x = x1, y = y1)]),
                    dict(data = [dict(x = x2, y = y2)])
                   ]
                  )

#### Plot the animated figure
In addition to the data and layout, there is a frames field which needs to be included

In [7]:
fig = dict(data=[{}], 
           layout=layout, 
           frames = frames)

offline.iplot(fig)

#### Adding a disconnected frame
So far, we have added frames which incrementally add to the existing plot. This produces the effect of a continuous line being drawn. However, if we add a frame which does not work off the existing line, but creates a separate one, then that continuous effect is lost

In [8]:
x3 = [2, 3]
y3 = [13, 14]

#### Redefine the frames list

In [9]:
frames = go.Frames([dict(data = [dict(x = x0, y = y0)]), 
                    dict(data = [dict(x = x1, y = y1)]),
                    dict(data = [dict(x = x2, y = y2)]),
                    dict(data = [dict(x = x3, y = y3)])
                   ]
                  )

#### Plot the new figure
As with any animation, if we want to display continuous motion, we need to ensure that the frames are set up correctly

In [10]:
fig = dict(data=[{}], 
           layout=layout, 
           frames = frames)

offline.iplot(fig)

### Animated view of average avocado price
Here, we use the avocados dataset to animate the average price of an organic California avocado over time. The source of the dataset is: <br />
https://www.kaggle.com/neuromusic/avocado-prices

In [11]:
data = pd.read_csv('datasets/avocado.csv')

#### We select only the data for organic California avocados
This is to simplify our plot

In [12]:
data_ca = data.loc[(data['region'] == 'California') & (data['type'] == 'organic')]
data_ca = data_ca.sort_values(by = ['Date'])
data_ca.head()

Index        Date  AveragePrice  Total Volume       4046      4225  \
9489     51  2015-01-04          1.24     142349.77  107490.73  25711.96   
9488     50  2015-01-11          1.10     158110.68  123712.51  25975.27   
9487     49  2015-01-18          1.24     195207.60  123138.29  58616.16   
9486     48  2015-01-25          1.30     121698.02   80952.24  28216.13   
9485     47  2015-02-01          1.11     196031.90  142332.45  41999.51   

      4770  Total Bags  Small Bags  Large Bags  XLarge Bags     type  year  \
9489  2.93     9144.15     9144.15         0.0          0.0  organic  2015   
9488  1.47     8421.43     8421.43         0.0          0.0  organic  2015   
9487  0.00    13453.15    13453.15         0.0          0.0  organic  2015   
9486  0.00    12529.65    12529.65         0.0          0.0  organic  2015   
9485  2.92    11697.02    11697.02         0.0          0.0  organic  2015   

          region  
9489  California  
9488  California  
9487  California  
9486  California  
9485  California

#### Load the dates and average prices into lists
We will be using these lists to generate the frames in our animation

In [13]:
avgPrices = list(data_ca['AveragePrice'])
dateList = list(data_ca['Date'])

#### Declare the lists we require to build our frames
We will iterate over the avocados data and append the date and prices to xList and yList respecively. We will then use the contents of xList and yList at each iteration to generate a frame which will be appended to framesList.

In [14]:
xList = []
yList = []
framesList = []

#### Generate the frames for the animation
In order to build a series of incrementally updated lists, we create a frame which contains all the x and y data of the previous frame plus a new x,y data point. 

We use the copy() method of a list which will create an independent (shallow) copy of the list for each frame. 

In [15]:
for i in range(len(dateList)):
    
    xList.append(dateList[i])
    yList.append(avgPrices[i])
    
    framesList.append(dict(data = [dict(x = xList.copy(), y = yList.copy())]))

#### Define the play button
This time we add a few more attributes to the Play button:
* <b>fromcurrent</b> will set whether hitting Play will resume animation from the current frame or from the beginning. We choose to resume from the current frame (which becomes significant since we're about to add a pause button)
* <b>transition</b> defines the transition between frames (the drawing of a line to the next point in our example)
* <b>frame</b> defines the state of the frame itself (between transitions)

We can speed up or slow down the animation by playing with the transition and frame durations.

In [16]:
playButton = dict(label = 'Play',
                  method= 'animate',
                  args= [None, 
                         dict(fromcurrent = True, 
                              transition = dict(duration = 200), 
                              frame = dict(duration = 100)
                             )
                        ]
                 )

#### Define a pause button
The arguments for a pause button include:
* an empty list [None] which effectively tells it to animate an empty sequence of frames
* setting the mode to 'immediate' which tells the plot to dump any queued frames from the animation

In [17]:
pauseButton = dict(label = 'Pause',
                  method= 'animate',
                  args= [[None], dict(mode = 'immediate')]
                 )

#### Define the layout
Here, we set the X axis to include the full range of date values in our dataset and leave some room on the Y axis for the average price.

We finally add the Play and Pause buttons

In [18]:
layout = go.Layout(xaxis = dict(range = [dateList[0], dateList[-1]]), 
                   yaxis = dict(range = [0, 1 + max(avgPrices)]),
                   
                   updatemenus = [dict(type = 'buttons',
                                       buttons = [playButton, pauseButton]
                                       )
                                 ]
                  )

#### Plot the figure
You can now play and pause the animation. Note that markers are drawn for the first few points

In [19]:
fig = dict(data=[{}], 
           layout=layout, 
           frames = framesList)

offline.iplot(fig)